# Функции серверного API

In [1]:
import dill
import json
import numpy as np
import pandas as pd

## Модель (pipeline)

Договорились о том, что первым в пайплайне стоит хелпер, в котором хранятся имена индекса и столбцов. Все имена нужны для работы модели.

In [2]:
with open("app/app/models/pipeline.dill", "rb") as f:
    pipeline = dill.load(f)

In [3]:
pipeline[0].frame_names

{'index': 'client_id', 'columns': ['trans_date', 'small_group', 'amount_rur']}

## Серверные функции

In [4]:
# pandas dataframe из request_json
def get_dataframe(request_json, pipeline):
    _names = pipeline[0].frame_names
    keys = [_names['index']] + _names['columns']
    frame_dict = {col: request_json[col] for col in keys}
    return pd.DataFrame(frame_dict).set_index(_names['index'])

## Исходные данные для тестирования

In [5]:
X_test = pd.read_pickle('data/X_test.pkl.zip')
y_test = pd.read_pickle('data/y_test.pkl.zip')

In [6]:
# Запрос на базе данных первых двух клиентов тестового датасета
first2 = X_test.index.unique()[:2]
request_json = X_test.loc[first2].reset_index().to_dict('list')

In [7]:
test_df = get_dataframe(request_json, pipeline)
test_df

,trans_date,small_group,amount_rur
client_id,,,
1049,1,31,37.781
1049,1,26,150.947
1049,1,11,19.234
1049,2,4,76.020
1049,2,24,267.391
...,...,...,...
11195,728,1,84.095
11195,728,11,56.741
11195,729,55,37.737


In [8]:
# client list
', '.join(map(str, test_df.index.unique()))

'1049, 11195'

In [9]:
y_pred = pipeline.predict(test_df)
response = {
    'success': True,
    'predictions': y_pred.tolist()
}

In [10]:
json.dumps(response)

'{"success": true, "predictions": [1, 1]}'